In [23]:
import numpy as np
import random
from sklearn import datasets
import pandas as pd

$Implementation$

In [24]:
def initialize_weight_matrix(x):
    w = []
    for i in range(1,len(x)):
        w.append(np.random.randn(len(x[i-1]),len(x[i])))
    return w

In [25]:
def feed_forward(w, x, y):
    # Feed Forward
    for i in range(1,len(x)):
        x[i] = np.matmul(w[i-1].T, y[i-1].T)
        y[i] = 1/(1+np.e**(-x[i]))

In [26]:
def backpropagate(w, x, y, d_out):
    # Backpropagation
    # Initial setup
    w_inter = []
    for i in range(1,len(x)):
        w_inter.append(np.random.randn(len(x[i-1]),len(x[i])))

    # Layer - 1
    for i in range(w[1].shape[0]):
        for j in range(w[1].shape[1]):
            w_inter[1][i][j] = w[1][i][j] - 0.001*(y[2][j]-d_out[j])*(y[2][j]*(1-y[2][j]))*y[1][i]

    # Layer - 2
    for h in range(w[0].shape[0]):
        for i in range(w[0].shape[1]):
            w_inter[0][h][i] = w[0][h][i] - 0.001*((y[2][0]-d_out[0])*(y[2][0]*(1-y[2][0]))*(w[1][i][0])*(y[1][i]*(1-y[1][i]))*(y[0][h]) +
                                            (y[2][1]-d_out[1])*(y[2][1]*(1-y[2][1]))*(w[1][i][1])*(y[1][i]*(1-y[1][i]))*(y[0][h]))

    return w_inter

In [27]:
iris = datasets.load_iris()
iris['data'][:5,:3]

array([[5.1, 3.5, 1.4],
       [4.9, 3. , 1.4],
       [4.7, 3.2, 1.3],
       [4.6, 3.1, 1.5],
       [5. , 3.6, 1.4]])

In [31]:
x = [np.zeros(3), np.zeros(4), np.zeros(2)]
w = initialize_weight_matrix(x)
for runs in range(600):
    for inp,out in list(zip(iris['data'][:100,:3],iris['target'][:100])):
        x = [np.array(inp), np.zeros(4), np.array(2)]                   # Inputs to neurons
        '''
        # To add new layer
        x.append(np.zeros(n))
        '''
        y = [np.array(inp), np.zeros(4), np.zeros(2)]                   # Output from neurons
        '''
        # To add new layer
        y.append(np.zeros(n))
        '''
        feed_forward(w,x,y)
        d_out = [1,0] if out==0 else [0,1]
        w = backpropagate(w,x,y,d_out)



In [32]:
w

[array([[-1.5587451 ,  0.68955177,  0.13508201,  1.29776239],
        [ 1.0866639 ,  0.32646727,  0.64767569, -0.27965854],
        [-0.03681665, -0.16395978,  1.06571191, -2.29590967]]),
 array([[-6.17860163e-03,  7.80906781e-01],
        [-1.40306791e+00,  5.24758018e-04],
        [ 7.53916154e-01,  5.61914563e-01],
        [ 1.51558040e+00, -1.38827149e+00]])]

In [35]:
x_pred = [iris['data'][13,:3],np.zeros(4),np.zeros(2)]
y_pred = x_pred
s_out = [1,0]
feed_forward(w,x_pred,y_pred)
y_pred[2]

array([0.67497077, 0.33633429])

$Verification$

In [36]:
iris = datasets.load_iris()
iris.keys()

dict_keys(['data', 'target', 'frame', 'target_names', 'DESCR', 'feature_names', 'filename', 'data_module'])

In [37]:
df = pd.DataFrame(data=iris['data'], columns=iris['feature_names'])
df['classes'] = iris['target']
df.head()

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),classes
0,5.1,3.5,1.4,0.2,0
1,4.9,3.0,1.4,0.2,0
2,4.7,3.2,1.3,0.2,0
3,4.6,3.1,1.5,0.2,0
4,5.0,3.6,1.4,0.2,0


In [38]:
df.shape

(150, 5)

In [39]:
df.drop('petal width (cm)',axis=1,inplace=True)
df = df[(df['classes'] == 0) | (df['classes'] == 1)]
df.shape,df.head()

((100, 4),
    sepal length (cm)  sepal width (cm)  petal length (cm)  classes
 0                5.1               3.5                1.4        0
 1                4.9               3.0                1.4        0
 2                4.7               3.2                1.3        0
 3                4.6               3.1                1.5        0
 4                5.0               3.6                1.4        0)

In [40]:
from sklearn.neural_network import MLPClassifier
ver_model = MLPClassifier(hidden_layer_sizes=(4,),solver='sgd', max_iter=600)

In [41]:
ver_model.fit(df.drop('classes',axis=1),y=df['classes'])

C:\Users\kishl\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\neural_network\_multilayer_perceptron.py:702: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (600) reached and the optimization hasn't converged yet.
  warnings.warn(


MLPClassifier(hidden_layer_sizes=(4,), max_iter=600, solver='sgd')

In [42]:
ver_model.coefs_

[array([[-0.20171404, -0.60254268, -0.00456867,  0.00722509],
        [-0.22525707, -0.50083807,  1.10702285, -0.91763517],
        [-0.32801113, -0.2217566 , -0.83534515,  0.06212787]]),
 array([[-0.38020621],
        [-0.51037681],
        [-1.47339927],
        [-0.12702933]])]

In [43]:
ver_model.predict_proba([list(df.iloc[13,:3])])

C:\Users\kishl\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but MLPClassifier was fitted with feature names
  warnings.warn(


array([[0.94580611, 0.05419389]])